In [10]:
import os
os.chdir("/gpfs/home/asun/jin_lab/baselines/0_datasets")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

/gpfs/group/jin/asun/baselines/0_datasets


In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import lightning as L

In [19]:
import importlib
importlib.reload(np)
import numpy as np

/gpfs/home/asun/miniforge3/envs/baselines/lib/python3.13/importlib/__init__.py:129: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  _bootstrap._exec(spec, module)


In [20]:
import scanpy as sc

ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

In [5]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import lightning as L

In [6]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))

    def forward(self, x):
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))

    def forward(self, x):
        return self.l1(x)

In [7]:
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset)

In [ ]:
# model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train model
trainer = L.Trainer()
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

In [ ]:
class PertDataModule(L.LightningDataModule):
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=64)

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import lightning as L
import numpy as np
import anndata as ad

# -----------------------------
# PyTorch Dataset wrapper
# -----------------------------
class AnnDataset(Dataset):
    def __init__(self, adata, idx, obs_cols=None, target_layer="X"):
        """
        adata: AnnData
        idx: indices (np.ndarray) for this split
        obs_cols: list of obs columns to include as covariates (optional)
        target_layer: which layer/matrix to use as y (default .X)
        """
        self.adata = adata
        self.idx = np.array(idx)
        self.obs_cols = obs_cols or []
        self.target_layer = target_layer

    def __len__(self):
        return len(self.idx)

    def __getitem__(self, i):
        i = self.idx[i]
        # Targets: expression matrix
        if self.target_layer == "X":
            y = self.adata.X[i].toarray().squeeze() if hasattr(self.adata.X, "toarray") else self.adata.X[i]
        else:
            y = self.adata.layers[self.target_layer][i]

        y = torch.tensor(y, dtype=torch.float32)

        # Optional covariates (categorical or numeric from obs)
        if self.obs_cols:
            obs_vals = [self.adata.obs[col].iloc[i] for col in self.obs_cols]
            x = torch.tensor(np.array(obs_vals), dtype=torch.float32)
        else:
            x = torch.tensor([])

        return x, y

# -----------------------------
# LightningDataModule
# -----------------------------
class AnnDataModule(L.LightningDataModule):
    def __init__(self, adata, batch_size=256, obs_cols=None, target_layer="X",
                 train_idx=None, val_idx=None, test_idx=None):
        super().__init__()
        self.adata = adata
        self.batch_size = batch_size
        self.obs_cols = obs_cols
        self.target_layer = target_layer
        self.train_idx = train_idx
        self.val_idx = val_idx
        self.test_idx = test_idx

    def setup(self, stage=None):
        # Default random split if none provided
        n = self.adata.n_obs
        if self.train_idx is None:
            idx = np.arange(n)
            np.random.shuffle(idx)
            n_train = int(0.8*n); n_val = int(0.1*n)
            self.train_idx, self.val_idx, self.test_idx = idx[:n_train], idx[n_train:n_train+n_val], idx[n_train+n_val:]

        self.ds_train = AnnDataset(self.adata, self.train_idx, self.obs_cols, self.target_layer)
        self.ds_val   = AnnDataset(self.adata, self.val_idx,   self.obs_cols, self.target_layer)
        self.ds_test  = AnnDataset(self.adata, self.test_idx,  self.obs_cols, self.target_layer)

    def train_dataloader(self):
        return DataLoader(self.ds_train, batch_size=self.batch_size, shuffle=True, num_workers=0)

    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=self.batch_size, shuffle=False, num_workers=0)

    def test_dataloader(self):
        return DataLoader(self.ds_test, batch_size=self.batch_size, shuffle=False, num_workers=0)

In [ ]:
boli = sc.read_h5ad('/gpfs/home/asun/jin_lab/baselines/0_datasets/boli_ctx.h5ad')
